# Kaggle Competition for Titanic Data #


In [ ]:
# Mount Google Drive and define time function #
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
COLAB = True
print("Note: using Google CoLab")

import time 

def time_elapsed(sec):
  h = int(sec/3600)
  m = int(sec/60)
  s = sec % 60
  return "{}:{>02}:{:>05.2f}".format(h,m,s)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab


In [ ]:
def prepare_df(df):
  # Create a Title Column #
  df.insert(2, "Title",'') 

  df.loc[df.Name.str.contains(pat = 'Mr'),'Title']='Mr' 
  df.loc[df.Name.str.contains(pat = 'Master'),'Title']='Master' 
  df.loc[df.Name.str.contains(pat = 'Miss'),'Title']='Miss' 
  df.loc[df.Name.str.contains(pat = 'Mrs'),'Title']='Mrs' 

  # filling age gaps #
  # Age depends on title and parch #

  # get median age of female childreen # Title Miss means unmarried woman and Parch > 0 means traveling with parents/childreen
  female_Miss_childreen = df[(df.Title=='Miss') & (df.Parch > 0) ]["Age"].median()
  female_Miss_adult = df[(df.Title=='Miss') & (df.Parch == 0) ]["Age"].median() # female adult without husband
  female_Mrs_adult = df[(df.Title=='Mrs')]["Age"].median() # female adult with husband
  male_Master = df[(df.Title=='Master')]["Age"].median() # male child
  male_Mr = df[(df.Title=='Mr')]["Age"].median()     
  overal_median = df["Age"].median()

  # fill those empty age gaps
  df.loc[(df.Title=='Miss') & (df.Parch > 0) & (df.Age.isnull()), "Age"] = female_Miss_childreen
  df.loc[(df.Title=='Miss') & (df.Parch == 0) & (df.Age.isnull()), "Age"] = female_Miss_adult
  df.loc[(df.Title=='Mrs') & (df.Age.isnull()), "Age"] = female_Mrs_adult
  df.loc[(df.Title=='Mr') & (df.Age.isnull()), "Age"] = male_Mr
  df.loc[(df.Title=='Master') & (df.Age.isnull()), "Age"] = male_Master
  df.loc[df.Age.isnull(), "Age"] =  overal_median

  # Age is not much important factor while you are above 18, so lets make column to separate kids from adults #
  # Column named "Adult" 1 means age is above 18, 0-belov
  df.insert(2, "Adult",'') 
  df.loc[(df.Age > 18), "Adult"] = 1.0
  df.loc[(df.Age <= 18), "Adult"] = 0.0
  df.loc[(df.Age.isnull()), "Adult"] = 1
  df["Adult"] = pd.to_numeric(df.Adult, errors='coerce') # to change from type obcjet to float

  # Important factor is to survive is Fare, more expensive ticket more chance to survive
  # but some people have the same ticket so need to divide their Fare to numbers of people sharing their ticket
  # then need to fill missing Fare gaps acording to Pclass they belong

  # Adding column Fare per Person #
  df.insert(12, "FarePP",0) 

  # Evaluating Fare PP #
  tickets_count = df.groupby("Ticket")["Ticket"].count() #counting tickets
  for ticket_number in tickets_count.keys():
    fpp = (df[df.Ticket == ticket_number]["Fare"]) / tickets_count[ticket_number]
    df.loc[df.Ticket == ticket_number, "FarePP"] = fpp

  # filling gaps
  FarePP_1_class = df[(df.Pclass==1) ]["FarePP"].median()
  FarePP_2_class = df[(df.Pclass==2) ]["FarePP"].median()
  FarePP_3_class = df[(df.Pclass==3) ]["FarePP"].median()

  df.loc[(df.Pclass==1) & (df.FarePP.isnull()), "FarePP"] = FarePP_1_class
  df.loc[(df.Pclass==2) & (df.FarePP.isnull()), "FarePP"] = FarePP_2_class
  df.loc[(df.Pclass==3) & (df.FarePP.isnull()), "FarePP"] = FarePP_3_class

  df = pd.concat([df,pd.get_dummies(df['Sex'],prefix="Sex")],axis=1)
  df.drop('Sex', axis=1, inplace=True)
  df = pd.concat([df,pd.get_dummies(df['Embarked'],prefix="Embarked")],axis=1)
  df.drop('Embarked', axis=1, inplace=True)

  df = pd.concat([df,pd.get_dummies(df['Title'],prefix="Title")],axis=1)
  df.drop('Title', axis=1, inplace=True)

  df['FarePP'] = zscore(df['FarePP'])
  df['Age'] = zscore(df['Age'])
  return df


In [ ]:
# Read Titanic Data #
import pandas as pd
from scipy.stats import zscore
import numpy as np
from tqdm import tqdm
import os 

DATA_PATH = '/content/drive/My Drive/projects/titanic'
TRAIN_FILE_PATH = os.path.join(DATA_PATH, 'train.csv') 
TEST_FILE_PATH = os.path.join(DATA_PATH, 'test.csv') 


df = pd.read_csv(TRAIN_FILE_PATH,na_values=['NA','?'])
df = prepare_df(df)

y = df['Survived'].values
x_columns = df.columns.drop('PassengerId').drop('Survived').drop('Ticket').drop('Fare').drop('Cabin').drop('Name')
x = df[x_columns].values

df_s = pd.read_csv(TEST_FILE_PATH,na_values=['NA','?'])
df_s = prepare_df(df_s)
x_columns_s = df_s.columns.drop('PassengerId').drop('Ticket').drop('Fare').drop('Cabin').drop('Name')

x_submit = df_s[x_columns_s].values






In [ ]:
# stworzenie własnej sieci neuronowej
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import math

def mlogloss(y_test, preds): # funckja 
    epsilon = 1e-15
    sum = 0
    for row in zip(preds,y_test):
        x = row[0][row[1]]
        x = max(epsilon,x)
        x = min(1-epsilon,x)
        sum+=math.log(x)
    return( (-1/len(preds))*sum)

def build_snn(input_size):
    neuronPct = 0.9652  # parametry zooptymalizowane za pomoca BayesianOplimization
    neuronShrink = 0.7319
    dropout = 0.01058
    lr = 0.08079
    neuronCount = int(neuronPct * 1000)   
    model = Sequential()
    layer = 0
    while neuronCount>25 and layer<4:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
            model.add(Dense(neuronCount,input_dim=input_size,activation=PReLU()))
        else:
            model.add(Dense(neuronCount, activation=PReLU())) 
        layer += 1
        # Add dropout after each hidden layer
        model.add(Dropout(dropout))
        # Shrink neuron count for each layer
        neuronCount = neuronCount * neuronShrink
    model.add(Dense(1,activation='sigmoid')) # Output
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=lr))
    return model

def blend_ensamble(x,y,x_submit):
  FOLDS = 5 # podzielenie danych treningowych na 10 czesci
  kf = StratifiedKFold(FOLDS,shuffle=True, random_state=0)
  folds = list(kf.split(x,y))

  models = [
            #SVC(kernel='rbf',shrinking=True,probability=True),
            KerasClassifier(build_fn = build_snn, input_size=x.shape[1]),
            KNeighborsClassifier(n_neighbors=5),
            RandomForestClassifier(n_estimators=200,n_jobs=-1,criterion='gini'),
            RandomForestClassifier(n_estimators=200,n_jobs=-1,criterion='entropy'),
            ExtraTreesClassifier(n_estimators=200,n_jobs=-1,criterion='gini'),
            ExtraTreesClassifier(n_estimators=200,n_jobs=-1,criterion='entropy'),
            GradientBoostingClassifier(learning_rate=0.05,subsample=0.5,max_depth=6,n_estimators=50)]

  dataset_blend_train = np.zeros((x.shape[0], len(models))) # każdy model wypracuje jedna wartosc dla jednej danej
  dataset_blend_test = np.zeros((x_submit.shape[0], len(models))) 

  for j, model in enumerate(models): # dla każdego modelu
    print(f'{j} Model: {model}')
    fold_sums = np.zeros((x_submit.shape[0],len(folds))) # dla każdego zestawu danych wyznaczone beda predyckje i wysrednione
    total_loss = 0
    for i, (train,test) in enumerate(folds):  # dla każdego zestawu danych treningowych
      x_train = x[train]
      y_train = y[train]
      x_test = x[test] # x testowe ale ze zbioru treningowego
      y_test = y[test]
      if j == 0:
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=10, verbose=0, mode='auto', restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train,y_train,validation_data=(x_test,y_test),
                  callbacks=[monitor],verbose=0,epochs=300)
      else:
        model.fit(x_train,y_train)
      pred = np.array(model.predict_proba(x_test))
      pred_submit = np.array(model.predict_proba(x_submit))
      dataset_blend_train[test,j] = pred[:,1] # zapisanie predykcji dla danego modelu oraz danego zestawu danych
      fold_sums[:,i] = pred_submit[:,1] # dla kazdego zestawu danych policzyc predykcje
      loss = mlogloss(y_test,pred) # policzenie pomocniczej funkcji celu
      total_loss+=loss # liczenie sumy funkcji celu dla modelu
      print(f'Fold {i} has loss: {loss}') 
    print("{}: Mean loss={}".format(model.__class__.__name__,total_loss/len(folds)))
    dataset_blend_test[:,j] = fold_sums.mean(1) # wyznaczenie jednej predykcji dla jednego modelu
  
  print('Blending Models')
  # wyznaczenie ostatecznej predykcji za pomoca regresji logistycznej
# ostateczne predykcja na podstawie predykcji wszystkich modeli
  return dataset_blend_train,dataset_blend_test
def stretch(y):
    return (y - y.min()) / (y.max() - y.min())

In [ ]:
(dataset_blend_train,dataset_blend_test) = blend_ensamble(x,y,x_submit)
#submit_data = stretch(submit_data)

blend = LogisticRegression(solver='lbfgs') 
blend.fit(dataset_blend_train,y) 
wynik=blend.predict_proba(dataset_blend_test) 

submit_data = stretch(wynik)
submit_data = np.round( submit_data[:, 1])
submit_data = submit_data.astype(int)
ids = [id+892 for id in range(len(submit_data))]
submit_filename = os.path.join(DATA_PATH, "Titanic_submit.csv")
submit_df = pd.DataFrame({'PassengerId': ids, 
                              'Survived': 
                              submit_data},
                             columns=['PassengerId',
                            'Survived'])
submit_df.to_csv(submit_filename, index=False)


0 Model: <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd012241470>
Fold 0 has loss: 0.6949571494494441
Fold 1 has loss: 0.5790863346383671
Fold 2 has loss: 0.8609926147730738
Fold 3 has loss: 0.6053200842146472
Fold 4 has loss: 0.7338222898233905
KerasClassifier: Mean loss=0.6948356945797846
1 Model: KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Fold 0 has loss: 1.6059721511101148
Fold 1 has loss: 1.1193560597653673
Fold 2 has loss: 1.65715687015957
Fold 3 has loss: 2.6461994230460353
Fold 4 has loss: 2.78477664181618
KNeighborsClassifier: Mean loss=1.9626922291794535
2 Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impu

In [ ]:
ids = [id+892 for id in range(len(submit_data))]
submit_filename = os.path.join(DATA_PATH, "Titanic_submit.csv")
submit_df = pd.DataFrame({'PassengerId': ids, 
                              'Survived': 
                              submit_data},
                             columns=['PassengerId',
                            'Survived'])
submit_df.to_csv(submit_filename, index=False)

In [ ]:
y_submition-submit_data

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0